# [Day 21](https://adventofcode.com/2020/day/21): Allergen Assessment

In [1]:
import re

with open("../data/21.txt", "r") as f:
    foods = re.findall(r"(.*) \(contains (.*)\)", f.read())
    foods = [(ing.split(), agn.split(", ")) for ing, agn in foods]
    inglists, agnlists = zip(*foods)

## Part 1

In [2]:
import numpy as np

def danger(agnlists, inglists):
    hasagn, _ = asmatrix(agnlists)
    hasing, ings = asmatrix(inglists)
    dangerous = section(maybedangerous(hasagn, hasing))
    return hasing, ings, dangerous

def asmatrix(xss):
    xs = sorted(set.union(*map(set, xss)))
    enum = {x: i for i, x in enumerate(xs)}
    mx = np.zeros((len(xs), len(xss)), dtype=bool)
    for j, _xs in enumerate(xss):
        mx[[enum[x] for x in _xs], j] = True
    return mx, np.array(xs)

def maybedangerous(hasagn, hasing):
    cnt = np.einsum("af,if->fai", hasagn, hasing).sum(axis=0)
    return cnt == cnt.max(axis=1, keepdims=True)

def section(x):
    s, sec = 0, np.empty((len(x), 2), dtype=int)
    ind = np.einsum("ijk->jki", np.indices(x.shape))
    while len(x):
        i = x.sum(axis=1) == 1
        j = x[i].any(axis=0)
        sec[s:(s := s + i.sum())] = ind[i][x[i]]
        x, ind = x[~i][:, ~j], ind[~i][:, ~j]
    return sec[:, 1][sec[:, 0].argsort()]

hasing, ings, dangerous = danger(agnlists, inglists)
assert 1882 == hasing.sum() - hasing[dangerous].sum()

## Part 2

In [3]:
canonicaldangerous = ",".join(ings[dangerous])
assert "xgtj,ztdctgq,bdnrnx,cdvjp,jdggtft,mdbq,rmd,lgllb" == canonicaldangerous